In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DRIVE_LOC = "/content/drive/MyDrive/mlforqc/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = xm.xla_device()

In [ ]:
class QuantumData(Dataset):
    
    def __init__(self, x_file, y_file):
        self.data_x = pd.read_csv(DRIVE_LOC+x_file)
        self.data_y = pd.read_csv(DRIVE_LOC+y_file)
        
        self.data_x = torch.tensor(self.data_x.values, dtype=torch.float32)#Keeping float64 was throwing some kind of error
        self.data_y = torch.tensor(self.data_y.values, dtype=torch.float32)
        
    def __len__(self):
        return len(self.data_x)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X = self.data_x[idx]
        y = self.data_y[idx]

        return X, y

In [ ]:
training_data = QuantumData('Dataset/Athens_x_train.csv', 'Dataset/Athens_y_train.csv')
testing_data = QuantumData('Dataset/Athens_x_test.csv', 'Dataset/Athens_y_test.csv')

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_loader = DataLoader(training_data, batch_size = BATCH_SIZE ,shuffle=True)
test_loader = DataLoader(testing_data, batch_size = 1,shuffle=True)

In [ ]:
class Slot(nn.Module):

  def __init__(self):
    super(Slot, self).__init__()

    self.slot_fc1 = nn.Linear(1024,256)
    self.slot_fc2 = nn.Linear(256,128)
    #The slot's output array contains no(processor qubits) + 1 
    #The last class is to show no mapping
    #ith index is one in the last layer of the slot implies 
    #Each slot_i shows which processor qubit j it should be mapped to in its array
    self.slot_fc3 = nn.Linear(128,5)

  def forward(self,x):
    x = self.slot_fc1(x)
    x = F.relu(x)
    
    x = self.slot_fc2(x)
    x = F.relu(x)
    x = self.slot_fc3(x)
    x = F.relu(x)

    return x

In [ ]:
class NeuralLayout(nn.Module):
    
    def __init__(self):
        super(NeuralLayout, self).__init__()

        self.dnn_fc1 = nn.Linear(78,264)
        self.dnn_fc1_bn = nn.BatchNorm1d(264)
        self.dnn_fc2 = nn.Linear(264,1024)
        self.dnn_fc2_bn = nn.BatchNorm1d(1024)
        self.dnn_dropout = nn.Dropout(p=0.5)
        
        self.slots = nn.ModuleList([Slot() for i in range(5)])
        
    def forward(self, x):
      
        x = self.dnn_fc1(x)
        x = self.dnn_fc1_bn(x)
        x = F.relu(x)
        x = self.dnn_fc2(x)
        x = self.dnn_fc2_bn(x)
        x = F.relu(x)
        x = self.dnn_dropout(x)

        out = []

        for slot in self.slots:
          out.append(slot(x))

        return tuple(out)

In [ ]:
nl = NeuralLayout().to(device=device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(nl.parameters(), lr=0.003)

In [ ]:
trained_till = 0
num_epochs = 100 - trained_till
#print(len(list(enumerate(train_loader))))
#nl.load_state_dict(torch.load(DRIVE_LOC+'Checkpoints/checkpoint'+str(trained_till)+'.pth'))

In [ ]:
for epoch in range(num_epochs):
    print("Epoch number ", epoch)
    for batch_idx, (data, targets) in enumerate((train_loader)):
        if targets.shape[0] != BATCH_SIZE:
              break
        nl.train()

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        #print(data.shape)
        #print(targets.shape)
        

        # Forward
        cross_entropy_loss = nn.CrossEntropyLoss()

        out = nl(data)

        #print(targets[0])

        batch_target_tensor = torch.zeros(BATCH_SIZE,5,5).to(device=device)

        for batch_entry in range(64):
          target = targets[batch_entry]

          final_target_tensor = torch.zeros(1,5,5)

          for j,i in enumerate(target):
            batch_target_tensor[batch_entry,j,int(i)] = 1


          #print(target)
          #print(batch_tensor[batch_entry,:,:])
          #print()

        #Shape for batch_tensor should be 64 x 5 (slot) x 5 (for each processor qubit)


    

        loss = None

        '''for i,out_i in enumerate(out):
          #print(i,out_i)
          slot_i_loss = loss_fn(out_i,batch_target_tensor[:,i,:])
          if not torch.any(slot_i_loss.isnan()):
            loss += slot_i_loss'''
        #print(loss_fn(out_i,batch_target_tensor[:,i,:]))
        #print(loss)

        #loss = sum([loss_fn(out_i,batch_target_tensor[:,i,:]) for i,out_i in enumerate(out)])

        #Dimension of out is 5x64x5

        restructured_out = torch.zeros(64,5,5).to(device=device)

        for i in range(len(out)):#Slot number
          for j in range(len(out[0])):#Batch number 
            for k in range(len(out[0][0])):#In slot index
              restructured_out[j,i,k] = out[i][j][k]

        
        # Backward
        optimizer.zero_grad()

        for i in range(5):
          if i == 0:
            loss = loss_fn(restructured_out[:,i,:],batch_target_tensor[:,i,:])
          else:
            loss += loss_fn(restructured_out[:,i,:],batch_target_tensor[:,i,:])

        print("Loss = "+str(loss.item()))
        loss.backward()

        '''loss0.backward(retain_graph = True)
        loss1.backward(retain_graph = True)
        loss2.backward(retain_graph = True)
        loss3.backward(retain_graph = True)
        loss4.backward()'''

        '''for i in range(5):
          l = loss_fn(out[i],batch_target_tensor[:,i,:])
          l.backward(retain_graph=(i != 4))
          print(l)'''

        '''loss = loss_fn(restructured_out,batch_target_tensor)
        loss.backward()
        print(loss)'''

        #loss.backward()
        #print(loss)

        # Gradient descent or adam step
        optimizer.step()


    torch.save(nl.state_dict(),DRIVE_LOC+'Checkpoints2/checkpoint'+str(epoch)+'.pth')
    

Streaming output truncated to the last 5000 lines.
Loss = 7.241271495819092
Loss = 7.239731788635254
Loss = 8.07485580444336
Loss = 7.138906955718994
Loss = 7.289327621459961
Loss = 6.748797416687012
Loss = 6.591613292694092
Loss = 6.900562286376953
Loss = 7.089146614074707
Loss = 7.543560981750488
Loss = 7.033109664916992
Loss = 7.143432140350342
Loss = 7.344991207122803
Loss = 7.604358196258545
Loss = 7.672209739685059
Loss = 7.973464012145996
Loss = 7.010621070861816
Loss = 7.056512832641602
Loss = 7.767795562744141
Loss = 7.481686592102051
Loss = 6.8468546867370605
Loss = 7.050501823425293
Loss = 7.188304901123047
Loss = 7.151516914367676
Loss = 7.804128646850586
Loss = 7.667359352111816
Loss = 7.797884941101074
Loss = 7.055683135986328
Loss = 6.962503433227539
Loss = 7.150547981262207
Loss = 6.975722789764404
Loss = 7.221492290496826
Loss = 7.283759117126465
Loss = 7.274521827697754
Loss = 7.409969806671143
Loss = 7.7227606773376465
Loss = 7.086501121520996
Loss = 7.77225780487060

In [ ]:
#print([i for i in nl.parameters()])

In [ ]:
correct_partial_mappings = 0
partial_mappings = 0
correct_mappings = 0
total_mappings = 0


for idx,(data,targets) in enumerate(test_loader):
    nl.eval()
    data = data.to(device=device)
    pred = nl(data)
    total_mappings += 1
    target_pred = [torch.argmax(pred[i]).item() for i in range(5)]
    if [torch.argmax(pred[i]).item() for i in range(5)] == targets.tolist():
      correct_mappings += 1
    target_opt = targets.tolist()[0]
    #print(target_opt[0])
    for i in range(5):
      if (target_pred[i]==target_opt[i]):
        correct_partial_mappings += 1
      partial_mappings += 1

print(correct_mappings)
print(total_mappings)
print(correct_partial_mappings)
print(partial_mappings)
print(correct_partial_mappings/partial_mappings)